# Seachable STAC catalog

This notebook shows an example how to access items and the corresponding hrefs stored in the assets for given search and open data parameters. A searchable STAC catalog implements the [STAC API - Item Search](https://github.com/radiantearth/stac-api-spec/tree/release/v1.0.0/item-search) conformance class, which provides the ability to search for STAC Item objects across collections.

### Setup
In order to run this notebook you need to install [`xcube`](https://xcube.readthedocs.io/en/latest/) and the [`xcube_stac`](https://github.com/xcube-dev/xcube-stac) plugin. You may install [`xcube_stac`](https://github.com/xcube-dev/xcube-stac) directly from the git repository by cloning the repository, directing into `xcube-stac`, and following the steps below:

```bash
conda env create -f environment.yml
conda activate xcube-stac
pip install .
```

Note that [`xcube`](https://xcube.readthedocs.io/en/latest/) is included in the `environment.yml`.  

Now, we first import everything we need:

In [1]:
from xcube.core.store import new_data_store, get_data_store_params_schema
import itertools

First, we get the store parameters needed to initialize a STAC [data store](https://xcube.readthedocs.io/en/latest/dataaccess.html#data-store-framework). 

In [2]:
store_params = get_data_store_params_schema("stac")
store_params

We determine the url of the [Earth Search](https://element84.com/earth-search/) STAC catalog and initiate a STAC [data store](https://xcube.readthedocs.io/en/latest/dataaccess.html#data-store-framework) where the `xcube-stac` plugin is recognized by setting the first argument to `"stac"` in the `new_data_store` function.

In [3]:
url = "https://earth-search.aws.element84.com/v1"
store = new_data_store("stac", url=url)

The data IDs point to a [STAC item's JSON](https://github.com/radiantearth/stac-spec/blob/master/item-spec/item-spec.md) and are specified by the segment of the URL that follows the catalog's URL. The data IDs can be streamed using the following code where we show the first 10 data IDs as an example.

In [4]:
data_ids = store.get_data_ids()
list(itertools.islice(data_ids, 10))

['collections/sentinel-1-grd/items/S1A_IW_GRDH_1SDV_20240612T105523_20240612T105544_054289_069A9A',
 'collections/sentinel-1-grd/items/S1A_IW_GRDH_1SDV_20240612T105458_20240612T105523_054289_069A9A',
 'collections/sentinel-1-grd/items/S1A_IW_GRDH_1SDV_20240612T105433_20240612T105458_054289_069A9A',
 'collections/sentinel-1-grd/items/S1A_IW_GRDH_1SDV_20240612T105408_20240612T105433_054289_069A9A',
 'collections/sentinel-1-grd/items/S1A_IW_GRDH_1SDV_20240612T105318_20240612T105343_054289_069A9A',
 'collections/sentinel-1-grd/items/S1A_IW_GRDH_1SDV_20240612T105253_20240612T105318_054289_069A9A',
 'collections/sentinel-1-grd/items/S1A_IW_GRDH_1SDV_20240612T105228_20240612T105253_054289_069A9A',
 'collections/sentinel-1-grd/items/S1A_IW_GRDH_1SDV_20240612T105203_20240612T105228_054289_069A9A',
 'collections/sentinel-1-grd/items/S1A_IW_GRDH_1SDV_20240612T105138_20240612T105203_054289_069A9A',
 'collections/sentinel-1-grd/items/S1A_IW_GRDH_1SDV_20240612T105113_20240612T105138_054289_069A9A']

In the next step, we can search for items using search parameters. The following code shows which search parameters are available.

In [5]:
search_params = store.get_search_params_schema()
search_params

Now, let's search for Sentinel-2 L2a data in the region of Lake Constance for the period from March 4th to March 6th, 2020.

In [6]:
descriptors = list(
    store.search_data(
        collections=["sentinel-2-l2a"],
        bbox=[9, 47, 10, 48],
        time_range=["2020-03-04", "2020-03-06"],
    )
)
[d.to_dict() for d in descriptors]

[{'data_id': 'collections/sentinel-2-l2a/items/S2A_32TMT_20200305_0_L2A',
  'data_type': 'dataset',
  'bbox': [7.662878883910047,
   46.85818510451771,
   9.130456971519783,
   47.85361872923358],
  'time_range': ['2020-03-05', None]},
 {'data_id': 'collections/sentinel-2-l2a/items/S2A_32TNT_20200305_0_L2A',
  'data_type': 'dataset',
  'bbox': [8.999746010269408,
   46.86543294418706,
   9.69101980741396,
   47.85369284462768],
  'time_range': ['2020-03-05', None]},
 {'data_id': 'collections/sentinel-2-l2a/items/S2A_32UMU_20200305_0_L2A',
  'data_type': 'dataset',
  'bbox': [7.639190643894996,
   47.75741268155721,
   9.132768981529203,
   48.752927528985374],
  'time_range': ['2020-03-05', None]},
 {'data_id': 'collections/sentinel-2-l2a/items/S2A_32UNU_20200305_0_L2A',
  'data_type': 'dataset',
  'bbox': [8.999741508947045,
   47.76332693643577,
   10.103302080212728,
   48.75300400802843],
  'time_range': ['2020-03-05', None]}]

In the next step, we can open the data for each data ID. The following code shows which parameters are available for opening the data.

In [7]:
open_params = store.get_open_data_params_schema()
open_params

We select the band red (B04), green (B03), and blue (B02) and lazily load the corresponding data. 

In [8]:
for descriptor in descriptors:
    ds = store.open_data(descriptor.data_id, asset_names=["blue", "green", "nir"])
    print("-" * 100)
    print(ds)

----------------------------------------------------------------------------------------------------
<xarray.Dataset> Size: 724MB
Dimensions:      (x: 10980, y: 10980)
Coordinates:
  * x            (x) float64 88kB 4e+05 4e+05 4e+05 ... 5.097e+05 5.098e+05
  * y            (y) float64 88kB 5.3e+06 5.3e+06 5.3e+06 ... 5.19e+06 5.19e+06
    spatial_ref  int64 8B 0
Data variables:
    blue         (y, x) uint16 241MB dask.array<chunksize=(512, 512), meta=np.ndarray>
    green        (y, x) uint16 241MB dask.array<chunksize=(512, 512), meta=np.ndarray>
    nir          (y, x) uint16 241MB dask.array<chunksize=(512, 512), meta=np.ndarray>
----------------------------------------------------------------------------------------------------
<xarray.Dataset> Size: 724MB
Dimensions:      (x: 10980, y: 10980)
Coordinates:
  * x            (x) float64 88kB 5e+05 5e+05 5e+05 ... 6.098e+05 6.098e+05
  * y            (y) float64 88kB 5.3e+06 5.3e+06 5.3e+06 ... 5.19e+06 5.19e+06
    spatial_ref  int6

This notebook will be continued once the data access is implemented.